In this notebook we'll look at a collaborative filtering 
based method of recommending relevant books to the user. 

In [ ]:
import os 
import numpy as np
import pandas as pd
import zipfile
import tensorflow as tf
from tensorflow import keras
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
os.getcwd()

'/content'

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


To have a look at the files within the zipfile.

In [ ]:
import zipfile as ZipFile
with zipfile.ZipFile('/content/gdrive/MyDrive/BookRec.zip', 'r') as zipObj:
   # Get list of files in zipfile
   listOfiles = zipObj.namelist()
   for elem in listOfiles:
      print(elem)

book1-100k.csv
book1000k-1100k.csv
book100k-200k.csv
book1100k-1200k.csv
book1200k-1300k.csv
book1300k-1400k.csv
book1400k-1500k.csv
book1500k-1600k.csv
book1600k-1700k.csv
book1700k-1800k.csv
book1800k-1900k.csv
book1900k-2000k.csv
book2000k-3000k.csv
book200k-300k.csv
book3000k-4000k.csv
book300k-400k.csv
book4000k-5000k.csv
book400k-500k.csv
book500k-600k.csv
book600k-700k.csv
book700k-800k.csv
book800k-900k.csv
book900k-1000k.csv
user_rating_0_to_1000.csv
user_rating_1000_to_2000.csv
user_rating_2000_to_3000.csv
user_rating_3000_to_4000.csv
user_rating_4000_to_5000.csv
user_rating_5000_to_6000.csv
user_rating_6000_to_11000.csv


Reading the contents of one of the files into a pandas dataframe. 

In [ ]:
zf = zipfile.ZipFile('/content/gdrive/MyDrive/BookRec.zip') 
df = pd.read_csv(zf.open('book1300k-1400k.csv'))
df.head()

,Id,Name,Authors,ISBN,Rating,PublishYear,PublishMonth,PublishDay,Publisher,RatingDist5,RatingDist4,RatingDist3,RatingDist2,RatingDist1,RatingDistTotal,CountsOfReview,Language,pagesNumber,Description,Count of text reviews
0,1300002,Still River,Harry Hunsicker,0312940904,3.69,2006,27,6,Minotaur Books,5:35,4:50,3:52,2:12,1:2,total:151,20,NaN,304,"<i>In a stark and haunting debut novel, Harry ...",20
1,1300003,The Naturalist on the River Amazons,Henry Walter Bates,1589761979,3.91,2002,15,8,Narrative Press,5:23,4:25,3:13,2:5,1:2,total:68,11,eng,432,Charles Darwin encouraged Bates to write this ...,11
2,1300017,Molecular Biology of Membrane Transport Disorders,Stanley G. Schultz,0306451646,0.00,1996,30,6,Springer,5:0,4:0,3:0,2:0,1:0,total:0,0,NaN,657,When the six of us gathered to start planning ...,0
3,1300027,The Question Of Discrimination: Racial Inequal...,Steven Shulman,0819552143,0.00,1989,1,9,Wesleyan,5:0,4:0,3:0,2:0,1:0,total:0,0,NaN,394,NaN,0
4,1300028,Unlevel Playing Fields,Randy Albelda,0070009686,3.58,1997,1,1,McGraw-Hill Companies,5:8,4:7,3:3,2:3,1:3,total:24,0,NaN,256,"Brief, topical, and issues-oriented, this inno...",0


To have a look at all the files in the zipfile, we read them into a temporary dataframe. 

In [ ]:

with zipfile.ZipFile('/content/gdrive/MyDrive/BookRec.zip', 'r') as zipObj:
   # Get list of files names in zipfile
   listOfbookfiles = zipObj.namelist()
   finaldf = pd.DataFrame()
   for elem in listOfbookfiles:           
        if elem.startswith('book'):
            df=pd.read_csv(zf.open(elem))
            print(df.shape)

(58292, 18)
(39705, 20)
(57046, 18)
(41892, 20)
(43622, 20)
(38288, 20)
(34759, 20)
(33439, 20)
(32986, 20)
(32105, 19)
(38863, 19)
(43561, 19)
(395957, 19)
(56182, 18)
(256595, 19)
(56586, 18)
(280256, 19)
(55155, 18)
(54859, 18)
(55156, 19)
(54273, 20)
(49843, 20)
(40890, 20)


Combine all the files that contain details abt the books into one master dataframe (books_df).

In [ ]:
with zipfile.ZipFile('/content/gdrive/MyDrive/BookRec.zip', 'r') as zipObj:
   # Get list of files names in zip
   listOfiles = zipObj.namelist()
   books_df = pd.DataFrame()
   for elem in listOfiles:           
        if elem.startswith('book'):
            df=pd.read_csv(zf.open(elem))
            books_df= pd.concat([df, books_df],axis=0)
books_df.shape

(1850310, 21)

To look at the first few rows of the master dataframe. 

In [ ]:
books_df.head(5)

,Id,Name,Authors,ISBN,Rating,PublishYear,PublishMonth,PublishDay,Publisher,RatingDist5,...,RatingDist3,RatingDist2,RatingDist1,RatingDistTotal,CountsOfReview,Language,pagesNumber,Description,Count of text reviews,PagesNumber
0,900000,Cabinet 03: Weather,Jeffrey Kastner,1932698019,4.00,2004,2,1,Cabinet,5:0,...,3:0,2:0,1:0,total:2,0,NaN,112.0,"Artwork by Joe Amrhein, Maria Blondeel, Spence...",0.0,NaN
1,900001,"Workouts in a Binder for Swimmers, Triathletes...",Nick Hansen,1931382743,4.48,2005,18,11,Velo Press,5:24,...,3:3,2:0,1:0,total:44,2,NaN,101.0,This waterproof book of 100 swimming workouts ...,2.0,NaN
2,900002,Las 120 jornadas de Sodoma,Marquis de Sade,8472233847,3.12,1992,1,3,TusQuets,5:1745,...,3:2935,2:1753,1:1408,total:10087,9,NaN,424.0,The 120 Days of Sodom by Marquis de Sade relat...,9.0,NaN
3,900004,The Traveler: An American Odyssey in the Himal...,Eric Hansen,0871563509,4.18,1995,6,6,"Random House, Inc.",5:6,...,3:3,2:0,1:0,total:17,1,NaN,102.0,This loving memorial to Swift is a moving phot...,1.0,NaN
4,900006,Vertical Smile,Richard Condon,0297994026,3.74,1972,20,4,Littlehampton Book Services Ltd,5:7,...,3:13,2:2,1:0,total:39,1,NaN,334.0,NaN,1.0,NaN


Cleaning up the data,addressing Nan values.

In [ ]:
books_df.drop(['PagesNumber'], axis=1, inplace=True)
books_df['pagesNumber'] = books_df['pagesNumber'].fillna(256)
books_df['Language'] = books_df['Language'].fillna('eng')

To replace all versions of English with one standard one. 

In [ ]:
books_df['Language']=books_df['Language'].replace({'en-US':'eng', 'en-GB':'eng', 'en-CA':'eng'})

In [ ]:
books_df['Language'].value_counts()

eng    1808160
fre      16327
ger      11470
spa       7245
jpn       2059
        ...   
myv          1
zul          1
nub          1
ada          1
myn          1
Name: Language, Length: 121, dtype: int64

We want to keep only English language books in our dataframe, so we'll remove those written in other languages eg. French,Japanese etc. 

In [ ]:
books_df = books_df[books_df['Language'] == 'eng']
books_df.shape

(1808160, 20)

To check that books in languages other than English have been removed. 

In [ ]:
books_df['Language'].value_counts()

eng    1808160
Name: Language, dtype: int64

To look at the user files in the zipfile

In [ ]:
import zipfile as ZipFile
with zipfile.ZipFile('/content/gdrive/MyDrive/BookRec.zip', 'r') as zipObj:
   # Get list of files names in zip
   listOfiles = zipObj.namelist()
   finaldf = pd.DataFrame()
   for elem in listOfiles:           
        if elem.startswith('user'):
            df=pd.read_csv(zf.open(elem))
            print(df.shape)
            

(51945, 3)
(42986, 3)
(30633, 3)
(46970, 3)
(46903, 3)
(15481, 3)
(127678, 3)


Combine all the files that contain details abt the users into a master dataframe (finaldf).

In [ ]:
import zipfile as ZipFile
with zipfile.ZipFile('/content/gdrive/MyDrive/BookRec.zip', 'r') as zipObj:
   # Get list of files names in zip
   listOfiles = zipObj.namelist()
   finaldf = pd.DataFrame()
   for elem in listOfiles:           
        if elem.startswith('user'):
            df=pd.read_csv(zf.open(elem))
            finaldf= pd.concat([df, finaldf],axis=0)
finaldf.shape

(362596, 3)

In [ ]:
finaldf.head(5)

,ID,Name,Rating
0,6675,"Baxter, the Pig Who Wanted to Be Kosher",it was amazing
1,6675,Set This House in Order,really liked it
2,6675,Paradise Park,really liked it
3,7027,Paradise Park,liked it
4,6675,The Dead Fish Museum,it was amazing


Using value counts on the ID column we can see the number of unique users in our dataset. 

In [ ]:
finaldf.ID.value_counts()

4196    3637
4806    3236
4693    2736
1134    2089
284     2052
        ... 
7399       1
5971       1
7400       1
7401       1
4828       1
Name: ID, Length: 8919, dtype: int64

Using value counts on the Name column we can see the number of unique books in our dataset. 

In [ ]:
finaldf.Name.value_counts()

Rating                                    4765
The Catcher in the Rye                     985
The Great Gatsby                           885
The Da Vinci Code (Robert Langdon, #2)     846
To Kill a Mockingbird                      830
                                          ... 
فریدریش نیچه (نسل قلم 84)                    1
ژان پل سارتر (نسل قلم 61)                    1
زیگموند فروید (نسل قلم 31)                   1
دل دلدادگی، جلد اول                          1
The Encantadas and Other Stories             1
Name: Name, Length: 103533, dtype: int64

The five types of ratings that the users can give to a book. 

In [ ]:
finaldf.Rating.value_counts()

really liked it                      132808
liked it                              96047
it was amazing                        92354
it was ok                             28811
did not like it                        7811
This user doesn't have any rating      4765
Name: Rating, dtype: int64

Dropping rows with no ratings. 

In [ ]:
index_names = finaldf[finaldf['Name'] == 'Rating'].index
finaldf.drop(index_names, inplace = True) 

We'll encode the ratings as categorical values ranging between 1-5, in the column Rating_Cat. 

In [ ]:
finaldf['Rating_Cat'] = finaldf['Rating']
finaldf['Rating_Cat']= finaldf['Rating_Cat'].replace({'did not like it': 1, 'it was ok': 2, 'liked it': 3, 'really liked it': 4, 'it was amazing': 5})
finaldf.drop(['Rating'],axis=1,inplace=True)
finaldf.rename({'ID': 'User_ID'}, axis=1, inplace=True)
finaldf.head()

,User_ID,Name,Rating_Cat
580,7064,Siddhartha,5
581,7077,Siddhartha,5
582,7112,Siddhartha,5
583,7126,Siddhartha,5
584,7175,Siddhartha,3


 Creating a dictionary(books_dict) which has names of books and their unique IDs stored as key value pairs.  






In [ ]:
books_list=finaldf['Name'].unique().tolist()
books_dict = dict()
i=100
for k in books_list:
    books_dict[k]=i
    i+=1


To have a look at five key-value pairs from the dictionary(books_dict) that we just created. 

In [ ]:
key_values_5 = {k: books_dict[k] for k in list(books_dict)[:5]}
key_values_5

{'Siddhartha': 100,
 'Fortunately, the Milk': 101,
 'Pleasure Activism: The Politics of Feeling Good': 102,
 'The Remarkable Inventions of Walter Mortinson': 103,
 'The Biology of Belief: Unleashing the Power of Consciousness, Matter and Miracles': 104}

Creating a column(Book_ID) with unique IDs for the books, as per the dictionary(books_dict).

In [ ]:
finaldf["Book_ID"] = finaldf["Name"].apply(lambda x: books_dict.get(x))
finaldf.head(5)

,User_ID,Name,Rating_Cat,Book_ID
580,7064,Siddhartha,5,100
581,7077,Siddhartha,5,100
582,7112,Siddhartha,5,100
583,7126,Siddhartha,5,100
584,7175,Siddhartha,3,100


We'll be keeping only those books that have been rated atleast 20 times, in order to avoid having a very sparse matrix.

In [ ]:
y = finaldf.groupby('Book_ID')['Rating_Cat'].count()>=20
frequent_books = y[y].index
final_df = finaldf[finaldf['Book_ID'].isin(frequent_books)]
final_df.shape

(140634, 4)

We can see we're left with around 2300 books. 

In [ ]:
final_df.Book_ID.value_counts()

845      976
1107     870
739      822
928      725
953      720
        ... 
2115      20
16044     20
10214     20
740       20
16327     20
Name: Book_ID, Length: 2336, dtype: int64

We'll remove the Users/User IDs that have rated less than 20 books too, to avoid sparsity. 
We'll also create a subset of users that have rated between 15-20 books which we'll use for evaluating the performance of our recommender system later.  

In [ ]:
ID_grp= final_df.groupby(['User_ID']).size().reset_index(name='counts')
x = final_df.groupby('User_ID')["Rating_Cat"].count() >= 20
x_test = final_df.groupby('User_ID')["Rating_Cat"].count().isin(range(15, 20))
regular_users = x[x].index
test_users = x_test[x_test].index

Creating a separate dataframe with just the test users, we can see we have around 250 users for testing. 

In [ ]:
filter_test = final_df[final_df['User_ID'].isin(test_users)]
filter_test.User_ID.value_counts()

2717    19
3420    19
8494    19
7       19
6       19
        ..
2856    15
9746    15
8513    15
6862    15
3142    15
Name: User_ID, Length: 249, dtype: int64

The dataframe of users that have rated atleast 20 books is obtained(filtered_rating), which will be used for training. 

In [ ]:
filtered_rating = final_df[final_df['User_ID'].isin(regular_users)]
filtered_rating.head(5)

,User_ID,Name,Rating_Cat,Book_ID
580,7064,Siddhartha,5,100
581,7077,Siddhartha,5,100
582,7112,Siddhartha,5,100
584,7175,Siddhartha,3,100
585,7206,Siddhartha,3,100


We have around 1600 users that we will be training our model on.  

In [ ]:
filtered_rating.User_ID.value_counts()

882      578
675      568
284      549
529      504
8835     503
        ... 
865       20
8641      20
713       20
10538     20
9304      20
Name: User_ID, Length: 1612, dtype: int64

In [ ]:
pd.set_option('display.max_rows', 1000)

Dropping the 'Name' column, which had the Book titles.  

In [ ]:
filtered_train = filtered_rating.reindex(['User_ID','Book_ID','Rating_Cat'], axis=1)
filtered_train.head()

,User_ID,Book_ID,Rating_Cat
580,7064,100,5
581,7077,100,5
582,7112,100,5
584,7175,100,3
585,7206,100,3


Creating a dataframe with Book_ID as the index, and User ids as the columns and rating values as cell entries. 

In [ ]:
pivot_df = filtered_train.pivot_table(index='Book_ID',columns='User_ID',values='Rating_Cat',fill_value=0)
pivot_df.shape

(2336, 1612)

Addressing Nan values 

In [ ]:
pivot_df.fillna(0, inplace = True)


In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=1000, random_state=20)
svd.fit(pivot_df)
result = svd.transform(pivot_df)

In [48]:
result.shape

(2336, 1000)

In [49]:
svd.explained_variance_ratio_.sum()

0.9747174617203953

Finding similarity scores on the basis of Cosine similarity. 

In [50]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_scores = cosine_similarity(result)

In [51]:
similarity_scores[5]

array([0.09954132, 0.23388216, 0.30367129, ..., 0.08799203, 0.01590945,
       0.03682339])

The dataframe with similarity scores of all books is obtained(book_sim_df), since the diagonal elements are similarity values of an individual book with itself we shall set it as 0. 

In [52]:
book_sim_df = pd.DataFrame(data=similarity_scores, index= pivot_df.index, columns= pivot_df.index)

In [53]:
book_sim_df.values[[np.arange(book_sim_df.shape[0])]*2] = 0
book_sim_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """Entry point for launching an IPython kernel.


Book_ID,100,114,116,123,129,130,131,132,133,134,...,29447,31219,32530,43399,43402,43436,43457,43505,47119,47154
Book_ID,,,,,,,,,,,,,,,,,,,,,
100,0.000000,0.131346,0.064414,0.011574,0.045442,0.099541,0.067457,0.075972,0.110851,0.103176,...,0.019539,0.044299,0.000350,0.092075,0.156903,0.113506,0.042575,0.052284,0.066464,0.132033
114,0.131346,0.000000,0.097513,0.127460,0.147702,0.233882,0.035899,0.081259,0.170538,0.121448,...,0.134094,0.021738,0.013514,0.103665,0.198948,0.076532,0.028565,0.032261,0.030846,0.026193
116,0.064414,0.097513,0.000000,0.206764,0.307643,0.303671,0.232987,0.067937,0.369014,0.069285,...,0.070120,0.001240,0.060070,0.182076,0.209343,0.100618,0.001386,0.175588,0.030406,0.119320
123,0.011574,0.127460,0.206764,0.000000,0.467667,0.345114,0.047113,0.039472,0.227081,0.062711,...,0.074643,0.001580,0.001986,0.088192,0.117259,0.042103,0.005251,0.028552,-0.001260,0.040023
129,0.045442,0.147702,0.307643,0.467667,0.000000,0.423374,0.120829,0.035014,0.347523,0.064003,...,0.148438,-0.000337,-0.000885,0.154026,0.186070,0.030913,0.007447,0.041889,-0.002650,0.058708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43436,0.113506,0.076532,0.100618,0.042103,0.030913,0.143566,0.066164,0.043793,0.089440,0.056418,...,0.011222,-0.002493,0.001520,0.093262,0.302536,0.000000,0.133848,0.108596,0.034835,0.046777
43457,0.042575,0.028565,0.001386,0.005251,0.007447,0.061928,0.036786,0.023768,0.029921,0.027087,...,0.029644,-0.001490,0.280748,0.045468,0.129645,0.133848,0.000000,0.154879,0.048335,0.028313
43505,0.052284,0.032261,0.175588,0.028552,0.041889,0.087992,0.172008,0.021145,0.113609,0.018472,...,0.019500,-0.000846,0.100859,0.162751,0.195023,0.108596,0.154879,0.000000,0.195571,0.227429


Function to obtain the book name from corresponding Book ID.

In [ ]:
def get_book_name(val):
    for key, value in books_dict.items():
         if value == val:
             return key
    return ("book for corresponding ID doesn't exist")

name=get_book_name(1486)
name

The function list_of_recs returns a list of 5 most similar books to a given book, based on the dataframe with similarity scores. 
The function final_recs_list returns a list of 5 recommendations for users based on 3 books that the user has previously liked.  

In [54]:
def list_of_recs(Book_ID, df=book_sim_df):
   a = df.sort_values(Book_ID, ascending=False).head(6)
   rec_list=[]
   i=0
   for i in range(0,5):
    rec_list.append(get_book_name(a.index.values[i]))
    i+=1
   return(rec_list)

def final_recs_list(bookids):
  recs_list=[]
  for i in bookids:    
      temp_list = list_of_recs(i)
      recs_list.extend(temp_list)

  recs_dict = {}
  for i in recs_list:
      recs_dict[i]=recs_dict.get(i,0)+1
    
  final_list=list(sorted(recs_dict, key=recs_dict.get, reverse=True)[:5])
  return final_list

The function evaluate_users takes a particular User ID from the dataframe of test users we had separated earlier(filter_test). It finds a list of 5 recommendations for the user on the basis of 3 books that the user has previously liked. 
The function match_percent,verifies whether the recommendations made for a given user match with any of the books that the user has read, hence giving us a way to evaluate the accuracy of our recommendations. 

In [55]:
def evaluate_users(id_code,filter_test):

  temp_df=filter_test.loc[filter_test['User_ID'] == id_code]
  user_df=temp_df.sort_values("Rating_Cat",ascending=False)
  user_df= user_df.head(3)
  bookids_list = user_df['Book_ID'].tolist()
  
  rec_list=[]
  rec_list=final_recs_list(bookids_list)
  match_val=0
  match_val=find_match_percent(rec_list,id_code,temp_df)
  return match_val

def find_match_percent(recs,user_id,test_user_df):
  books_list = test_user_df['Name'].tolist()
  match = 0
  for x in books_list:
    match+= recs.count(x)
  return ((match/5)*100)


The following code block calculates the overall percentage of matches for all our test users.

In [62]:
test_users_list= filter_test.User_ID.unique().tolist()
match=0
total=0
for x in test_users_list:
  match=evaluate_users(x,filter_test) 
  total+= match
print ("Overall match percentage =", total/249)


Overall match percentage = 25.38152610441767


In [ ]:
# filter_test[(filter_test.User_ID == 10118)]